In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
%aimport utils, Networks

In [3]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import random
from utils import plot_images, load_embeddings_and_ids
from Networks import ContentBasedLearn2RankNetwork_Evaluation

In [4]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

###  Load pre-trained image embeddings

In [5]:
youtube_like = load_embeddings_and_ids(
('/mnt/workspace/Ugallery/Youtube-like/v34(10M-500K,avg+max,rsnt50+rsnxt101,u(300,300,200)i(200,200),hyb(fa+dnn),rl(full),ab.2,cm.18,fg.7,wd.0001)/'),
'item_vectors.npy', 'ids')

In [6]:
youtube_like_embeddings = youtube_like['featmat']
artwork_ids = youtube_like['index2id']
artwork_id2index = youtube_like['id2index']

In [7]:
n_artworks = len(artwork_ids)
n_artworks

13297

In [8]:
youtube_like_embeddings.shape

(13297, 200)

### Testing Network

In [9]:
MODEL_PATH = '/mnt/workspace/pamessina_models/ugallery/youtube_like/v34(10M-500K,avg+max,rsnt50+rsnxt101,u(300,300,200)i(200,200),hyb(fa+dnn),rl(full),ab.2,cm.18,fg.7,wd.0001)/'
MODEL_PATH

'/mnt/workspace/pamessina_models/ugallery/youtube_like/v34(10M-500K,avg+max,rsnt50+rsnxt101,u(300,300,200)i(200,200),hyb(fa+dnn),rl(full),ab.2,cm.18,fg.7,wd.0001)/'

In [10]:
image_cache = dict()

In [11]:
all_indexes = list(range(n_artworks))

In [12]:
def get_recommendation(profile_indexes, topk=10):    
#     candidate_indexes = set(random.sample(all_indexes, 10000))
#     candidate_indexes.update(profile_indexes)
#     candidate_indexes = list(candidate_indexes)
    candidate_indexes = all_indexes
    with tf.Graph().as_default():      
        network = ContentBasedLearn2RankNetwork_Evaluation(
            user_layer_units=[300,300,200],
            latent_space_dim=200,
            profile_pooling_mode='AVG+MAX',
        )
        with tf.Session() as sess:
            saver = tf.train.Saver()            
            saver.restore(sess, tf.train.latest_checkpoint(MODEL_PATH))
            match_scores = network.get_match_scores(sess, youtube_like_embeddings, profile_indexes, candidate_indexes)
            tuples = [(s,i) for i,s in zip(candidate_indexes, match_scores)]
            tuples.sort(reverse=True)
            return [tuples[i][1] for i in range(topk)]

In [51]:
profile_indexes = random.sample(all_indexes, 3)
# profile_indexes = [artwork_id2index[45020]]
# profile_indexes = [artwork_id2index[46314]]
# profile_indexes = [artwork_id2index[53547]]
# profile_indexes = [artwork_id2index[56625]]
# profile_indexes = [artwork_id2index[37735]]
# profile_indexes = [artwork_id2index[59149]]
# profile_indexes = [artwork_id2index[36604]]
# profile_indexes = [artwork_id2index[28839]]
# profile_indexes = [artwork_id2index[53089]]
# profile_indexes = [artwork_id2index[48587]]
# profile_indexes = [artwork_id2index[31245]]
profile_ids = [artwork_ids[i] for i in profile_indexes]
rec = get_recommendation(profile_indexes, topk=20)
rec_ids = [artwork_ids[i] for i in rec]
print('profile_indexes = ', profile_indexes)
print('profile_ids = ', profile_ids)
print('recommendation: ', rec_ids)

INFO:tensorflow:Restoring parameters from /mnt/workspace/pamessina_models/ugallery/youtube_like/v34(10M-500K,avg+max,rsnt50+rsnxt101,u(300,300,200)i(200,200),hyb(fa+dnn),rl(full),ab.2,cm.18,fg.7,wd.0001)/
profile_indexes =  [1961, 4346, 653]
profile_ids =  [38952, 44999, 34966]
recommendation:  [38952, 34966, 42436, 31793, 40391, 52312, 42095, 38445, 35456, 50704, 32053, 43902, 47318, 43923, 37138, 42501, 24795, 47949, 56383, 33654]


In [53]:
# print('-------------------------------')
# plot_images(plt, image_cache, profile_ids)
# print('-------------------------------')
# plot_images(plt, image_cache, rec_ids)